In [1]:
import wandb
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import LinearLR, CosineAnnealingLR, SequentialLR
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.models.convnext import LayerNorm2d

import torchvision
from torchvision import datasets, models
from torchvision import transforms
from torchvision.transforms import (Compose, RandomResizedCrop, 
                                    RandomHorizontalFlip, ColorJitter, 
                                    AutoAugment, InterpolationMode, v2, ToTensor)
from torchvision.io import read_image
from torchvision.models.convnext import ConvNeXt_Tiny_Weights

import matplotlib.pyplot as plt
import os
import pandas as pd
import glob
import numpy as np
import cv2
import random
import PIL
import seaborn as sn

from torch_ema import ExponentialMovingAverage
from tqdm.auto import tqdm
import time 

import pytorch_lightning as pl

from torchmetrics import Accuracy

In [2]:
# Hyperparameters
shape = (224, 224)
epochs=50
num_classes=15
batch_size=8
learning_rate=5e-5
weight_decay = 1e-8
optimizer_momentum = (0.9, 0.999)
scale_factor = 0.8 
drop_path_rate = 0.1
label_smoothing = 0.1
num_workers = 4

# Model

# Train

In [5]:
train_dir = "imagens/swedish/train"
test_dir = "imagens/swedish/test"

In [6]:
data_module = CustomImageModule(train_dir, test_dir, batch_size, num_workers)
model = CustomConvNeXtTiny()

In [7]:
data_module

In [8]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.models.convnext as ConvNeXt
from torchvision.models import ConvNeXt_Tiny_Weights
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
from pytorch_lightning.utilities.types import TRAIN_DATALOADERS
from torchmetrics import Accuracy
from pytorch_lightning import Trainer

In [9]:
# Crie o Trainer e execute o treinamento
trainer = Trainer(accelerator="gpu", devices=1, precision='16-mixed', max_epochs=10)  # Ajuste o número de GPUs conforme necessário
trainer.fit(model, data_module)
trainer.test(model, data_module)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/rodrigo/miniconda3/envs/mestrado/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type               | Params | Mode 
-----------------------------------------------------------------
0 | model             | ConvNeXt           | 27.8 M | train
1 | sequential_layers | Sequential         | 13.1 K | train

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_accuracy_epoch      0.9777777791023254
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_accuracy_epoch': 0.9777777791023254}]

In [10]:
data_module.setup('test')